In [19]:
from requests_ratelimiter import LimiterSession
import sys
import sqlite3
import pandas as pd

In [20]:
# TODOS
#   see if discussion and comment data play nice with sqlite database
#   problem: do we have enough space to keep all the records?
#   problem: how to iteratively pull all the comments/discussions into the database 
#      without repeats or overwhelming the api
#   question: possible to set up web hook to get new data as it is posted?

In [21]:
def size_in_mb(obj):
    '''get the size of the given object in megabytes'''
    return sys.getsizeof(obj)/1024/1024


In [22]:
def endpoint_size_estimator(api_session, api_base_url, endpoint, record_limit=500):
    '''Estimate the amount of data present at an API endpoint. 
    Intended for use with the Vanilla forum api but may generalize'''
    size = 0
    # get the first page
    response = api_session.get(api_base_url+endpoint+"?"+f"limit={record_limit}")
    size += size_in_mb(response.content)
    # get the remaining pages
    while "x-app-page-next-url" in response.headers.keys():
        response = api_session.get(response.headers['x-app-page-next-url'])
        size += size_in_mb(response.content)
        print(size, response.headers['x-app-page-next-url'])
    return size
    

In [23]:
def get_all_from_day():
    pass

In [24]:
def get_all():
    pass

In [25]:
per_minute_limit = 100.
api_base_url = "https://forums.elderscrollsonline.com/api/v2/"
session = LimiterSession(per_minute = per_minute_limit)

In [26]:
response = session.get(api_base_url+"comments?page=100&limit=100")
# size = endpoint_size_estimator(session, api_base_url, 'comments', record_limit=100)
# print(size)

In [27]:
print(response.content)
comments_df = pd.DataFrame(response.json())
comments_df = comments_df.drop(labels=["image", "insertUser","attributes"],axis="columns")
comments_df

b'[\n    {\n        "commentID": 497925,\n        "discussionID": 61051,\n        "parentRecordType": "discussion",\n        "parentRecordID": 61051,\n        "name": "Re: 2-3 second freezes",\n        "categoryID": 277,\n        "body": "Awesomonium Process is the thing taking up all my CPU",\n        "dateInserted": "2014-03-31T19:44:30+00:00",\n        "dateUpdated": null,\n        "insertUserID": 379598,\n        "updateUserID": null,\n        "score": 0,\n        "depth": 1,\n        "scoreChildComments": 0,\n        "countChildComments": 0,\n        "insertUser": {\n            "userID": 379598,\n            "name": "alewis478b14_ESO",\n            "url": "https://forums.elderscrollsonline.com/profile/alewis478b14_ESO",\n            "photoUrl": "https://us.v-cdn.net/5020507/uploads/bf47ba6b81f347a352defdda0e8d80d5.png",\n            "dateLastActive": "2020-06-10T04:50:20+00:00",\n            "banned": 0,\n            "punished": 0,\n            "private": false,\n            "lab

,commentID,discussionID,parentRecordType,parentRecordID,name,categoryID,body,dateInserted,dateUpdated,insertUserID,updateUserID,score,depth,scoreChildComments,countChildComments,url,type,format,answerStatusID
0,497925,61051,discussion,61051,Re: 2-3 second freezes,277,Awesomonium Process is the thing taking up all...,2014-03-31T19:44:30+00:00,None,379598,NaN,0,1,0,0,https://forums.elderscrollsonline.com/en/discu...,comment,BBCode,NaN
1,497926,61904,discussion,61904,Re: Well done Zenimax,352,I was a little upset when I saw some of the qu...,2014-03-31T19:44:41+00:00,None,4743204,NaN,0,1,0,0,https://forums.elderscrollsonline.com/en/discu...,comment,BBCode,NaN
2,497927,62938,discussion,62938,Re: Why limit the vanity pets and treasure map...,195,I admit.. if I had known it was just for one c...,2014-03-31T19:44:43+00:00,None,20204,NaN,2,1,0,0,https://forums.elderscrollsonline.com/en/discu...,comment,BBCode,NaN
3,497928,63003,discussion,63003,"Re: How ""Increase inspiration gained from extr...",185,It definitely means that you will get X% more ...,2014-03-31T19:44:47+00:00,None,661093,NaN,2,1,0,0,https://forums.elderscrollsonline.com/en/discu...,comment,BBCode,NaN
4,497929,62603,discussion,62603,Re: Deleted Imperial Edition/Early Access Items,195,I've already opened a case with the support bu...,2014-03-31T19:45:15+00:00,2014-03-31T19:54:06+00:00,2174987,2174987.0,1,1,0,0,https://forums.elderscrollsonline.com/en/discu...,comment,BBCode,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,498026,63035,discussion,63035,Re: Proving trust bugged?,195,"Yay it works!, and thanks <img class=""emoji"" s...",2014-03-31T20:08:03+00:00,None,4714872,NaN,0,1,0,0,https://forums.elderscrollsonline.com/en/discu...,comment,BBCode,NaN
96,498027,60955,discussion,60955,Re: Disconnecting with an 'Unknown Error' mess...,195,What I don't understand is that if this is ISP...,2014-03-31T20:08:07+00:00,None,4735517,NaN,0,1,0,0,https://forums.elderscrollsonline.com/en/discu...,comment,BBCode,NaN
97,498028,62913,discussion,62913,Re: looking for guild,265,Says your alliance is ebonheart pact? I'm with...,2014-03-31T20:08:10+00:00,None,4744546,NaN,0,1,0,0,https://forums.elderscrollsonline.com/en/discu...,comment,BBCode,NaN
98,498029,62967,discussion,62967,Re: World freezing w/ long lag spike or unknow...,195,"\n<blockquote class=""Quote UserQuote blockquot...",2014-03-31T20:08:48+00:00,None,4748013,NaN,1,1,0,0,https://forums.elderscrollsonline.com/en/discu...,comment,BBCode,NaN


In [22]:
# estimate size / 10,000 comments per results 15mb/100 pgs
# 6000 pages of general discussions, 30 disussions / page
num_discussions = 6000*30
mb_per_comment = 15/10000
comments_per_discussion = 60  # hopefully generous estimate

In [23]:
num_discussions*mb_per_comment*comments_per_discussion

16200.0

In [25]:
# database stuff
con = sqlite3.connect("data/eso_forum.db")
cur = con.cursor()  # database cursor
comments_df.to_sql("comments", con, index=False, if_exists="append")

100

In [16]:
response.headers

{'Date': 'Wed, 22 Oct 2025 18:43:26 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'CF-RAY': '992b23767defa21f-MSP', 'x-lookup-mode': 'normal', 'x-app': '4381ad28655b6c0ddcb4c76a1583cf579bf08e02/cl53204', 'p3p': 'CP="CAO PSA OUR"', 'x-request-id': '992b23767defa21f-SEA', 'access-control-expose-headers': 'Authorization, Content-Type, x-app-page-current, x-app-page-first-url, x-app-page-last-url, x-app-page-limit, x-app-page-next-url, x-app-page-prev-url, x-app-page-result-count', 'Cache-Control': 'public, max-age=120', 'vary': 'Accept-Encoding, Cookie', 'expires': 'Wed, 22 Oct 2025 18:45:26 GMT', 'vdk': '68f908b6', 'x-vanilla-version': '2025.010', 'content-security-policy': "script-src 'nonce-286347d37d0925b296f9a9c6eb3a28d7' 'unsafe-inline' 'self' blob: https://developers.panopto.com https://embed-cdn.gettyimages.com https://s.imgur.com https://platform.instagram.com https://platform.twitter.com https://cdn.syndicati

In [28]:
response.json()

[{'commentID': 497925,
  'discussionID': 61051,
  'parentRecordType': 'discussion',
  'parentRecordID': 61051,
  'name': 'Re: 2-3 second freezes',
  'categoryID': 277,
  'body': 'Awesomonium Process is the thing taking up all my CPU',
  'dateInserted': '2014-03-31T19:44:30+00:00',
  'dateUpdated': None,
  'insertUserID': 379598,
  'updateUserID': None,
  'score': 0,
  'depth': 1,
  'scoreChildComments': 0,
  'countChildComments': 0,
  'insertUser': {'userID': 379598,
   'name': 'alewis478b14_ESO',
   'url': 'https://forums.elderscrollsonline.com/profile/alewis478b14_ESO',
   'photoUrl': 'https://us.v-cdn.net/5020507/uploads/bf47ba6b81f347a352defdda0e8d80d5.png',
   'dateLastActive': '2020-06-10T04:50:20+00:00',
   'banned': 0,
   'punished': 0,
   'private': False,
   'label': '✭✭✭',
   'labelHtml': '✭✭✭'},
  'url': 'https://forums.elderscrollsonline.com/en/discussion/comment/497925#Comment_497925',
  'type': 'comment',
  'format': 'BBCode',
  'attributes': {}},
 {'commentID': 497926,
